### Context in Tools (Making Tools Stateful & Aware)

**Key Concept**: Tools become truly powerful when they can access runtime state, configuration, and memory. The `ToolRuntime` pattern provides a unified way to inject dependencies into tools.

**What this covers:**
1. Why context matters for tools
2. Understanding ToolRuntime (state, context, store, config)
3. Building context-aware tools
4. Practical patterns: user preferences, conversation memory, streaming
5. Comparison with older injection patterns

In [2]:
from dataclasses import dataclass
from langchain.agents import create_agent
from langchain.tools import tool, ToolRuntime
from langchain_groq import ChatGroq

from typing import Annotated
import json

/Users/yashsarode/Downloads/Personal Projects/Python/LangGraph-personal/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
llm = ChatGroq(
    model="meta-llama/llama-4-maverick-17b-128e-instruct", 
    temperature=0,  # Deterministic for consistent tool usage
)

### Why Context Matters

**Problem**: Tools often need information that shouldn't go through the LLM:
- User IDs (privacy/security)
- Database connections (technical details)
- API keys (security)
- Session state (efficiency)

**Bad approach**: "Hey model, here's user_id=123, remember to include it in tool calls"
- Wastes tokens
- Security risk
- Unreliable

**Good approach**: Inject context directly into tools at runtime
- LLM never sees sensitive data
- Tools get exactly what they need
- More testable and maintainable

### Understanding ToolRuntime

`ToolRuntime` is a special parameter that LangChain automatically injects into tools. It provides access to:

- **State**: Mutable data flowing through execution (messages, counters)
- **Context**: Immutable configuration (user IDs, session details)
- **Store**: Long-term persistent memory
- **Config**: Runtime configuration
- **Tool Call ID**: Current execution identifier

Think of it as a "dependency injection container" for tools.

In [4]:
# Accessing state
from langchain.tools import tool, ToolRuntime

# Access the current conversation state
@tool
def summarize_conversation(
    runtime: ToolRuntime
) -> str:
    """Summarize the conversation so far."""
    messages = runtime.state["messages"]

    human_msgs = sum(1 for m in messages if m.__class__.__name__ == "HumanMessage")
    ai_msgs = sum(1 for m in messages if m.__class__.__name__ == "AIMessage")
    tool_msgs = sum(1 for m in messages if m.__class__.__name__ == "ToolMessage")

    return f"Conversation has {human_msgs} user messages, {ai_msgs} AI responses, and {tool_msgs} tool results"

# Access custom state fields
@tool
def get_user_preference(
    pref_name: str,
    runtime: ToolRuntime  # ToolRuntime parameter is not visible to the model
) -> str:
    """Get a user preference value."""
    preferences = runtime.state.get("user_preferences", {})
    return preferences.get(pref_name, "Not set")

# NOTE: The runtime parameter is hidden from the model. For the example above, the model only sees pref_name in the tool schema - runtime is not included in the request.

- `@dataclass` decorator (from the built-in dataclasses module) provides an easy way to define classes whose main job is to hold data.
- `@dataclass` is used to define UserContext cleanly, eliminating boilerplate, making the code concise and expressive (just data).
- `ToolRuntime[UserContext]` uses Python’s generic typing system to specify that the tool’s runtime context will be of type UserContext, enabling type safety, clarity, and IDE/type-checker support.

In [7]:
from dataclasses import dataclass
from langchain.agents import create_agent
from langchain.tools import tool, ToolRuntime


USER_DATABASE = {
    "user123": {
        "name": "Alice Johnson",
        "account_type": "Premium",
        "balance": 5000,
        "email": "alice@example.com"
    },
    "user456": {
        "name": "Bob Smith",
        "account_type": "Standard",
        "balance": 1200,
        "email": "bob@example.com"
    }
}

@dataclass
class UserContext:
    user_id: str

@tool
def get_account_info(runtime: ToolRuntime[UserContext]) -> str:
    """Get the current user's account information."""
    user_id = runtime.context.user_id

    if user_id in USER_DATABASE:
        user = USER_DATABASE[user_id]
        return f"Account holder: {user['name']}\nType: {user['account_type']}\nBalance: ${user['balance']}"
    return "User not found"

agent = create_agent(
    llm,
    tools=[get_account_info],
    context_schema=UserContext,
    system_prompt="You are a financial assistant."
)

result = agent.invoke(
    {"messages": [{"role": "user", "content": "What's my current balance?"}]},
    context=UserContext(user_id="user123")
)

print(result["messages"][-1].pretty_print())

================================== Ai Message ==================================

Your current balance is $5000.
None



### Memory (Store)
Access persistent data across conversations using the store. 
The store is accessed via `runtime.store` and allows you to save and retrieve user-specific or application-specific data.
Tools can access and update the store through `ToolRuntime`

In [9]:
from typing import Any
from langgraph.store.memory import InMemoryStore
from langchain.agents import create_agent
from langchain.tools import tool, ToolRuntime


# Access memory
@tool
def get_user_info(user_id: str, runtime: ToolRuntime) -> str:
    """Look up user info."""
    store = runtime.store
    user_info = store.get(("users",), user_id)
    return str(user_info.value) if user_info else "Unknown user"

# Update memory
@tool
def save_user_info(user_id: str, user_info: dict[str, Any], runtime: ToolRuntime) -> str:
    """Save user info."""
    store = runtime.store
    store.put(("users",), user_id, user_info)
    return "Successfully saved user info."

store = InMemoryStore()
agent = create_agent(
    llm,
    tools=[get_user_info, save_user_info],
    store=store
)

# First session: save user info
result = agent.invoke({
    "messages": [{"role": "user", "content": "Save the following user: userid: abc123, name: Foo, age: 25, email: foo@langchain.dev"}]
})
print(result["messages"][-1].pretty_print())

# Second session: get user info
result = agent.invoke({
    "messages": [{"role": "user", "content": "Get user info for user with id 'abc123'"}]
})
# Here is the user info for user with ID "abc123":
# - Name: Foo
# - Age: 25
# - Email: foo@langchain.dev

print(result["messages"][-1].pretty_print())

================================== Ai Message ==================================

The user info for userid "abc123" has been successfully saved.
None
================================== Ai Message ==================================

The user with id 'abc123' is Foo, aged 25, and their email is foo@langchain.dev.
None
